## Descriptive statistics for flagged revisions in German Wikipedia

What % of revisions are flagged? For users with under 200 edits. For users with more than 200 edits. 
https://de.wikipedia.org/wiki/Spezial:Letzte_%C3%84nderungen?hidebots=1&hidecategorization=1&hideWikibase=1&flaggedrevs=reviewed&limit=50&days=7&urlversion=2

There is a table for flaggedrevs, but I'm not sure which fields indicate that 'flaggedrevs=reviewed'.

In [1]:
%%capture
!pip install mwclient
!pip install mwviews
!pip install mwapi
!pip install mwreverts

def wmftimestamp(bytestring):
    if bytestring:
        s = bytestring.decode('utf-8')
        return dt.strptime(s, '%Y%m%d%H%M%S')
    else:
        return bytestring
    

def decode_or_nouser(b):
    return b.decode('utf-8') if b else '#nosuchuser'

!mkdir outputs
!mkdir checkpoints


import json
import requests

from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError
from pymysql.err import InternalError, OperationalError
import sys, os
import pandas as pd
import numpy as np

import mwclient
import mwviews
import mwapi
import mwreverts

from datetime import datetime as dt
from datetime import timedelta as td

%pylab inline

In [3]:
import mwreverts.api
import mwapi

# We'll use the mwreverts API check.  In order to do that, we need an API session
de_session = mwapi.Session("https://de.wikipedia.org", 
                        user_agent="Max Klein Newcomer Quality <max@notconfusing.com>")

# for a single user, get their revisions and determine how many are flagged or not

def get_revisions_and_flagged_data(user_id, treatment_date):
    rev_flag_sql = """
        select rev_id, 
                rev_page, 
                page_namespace, 
                rev_timestamp, 
                fr_timestamp, 
                (select max(fr_timestamp) from flaggedrevs where fr_page_id=rev_page and fr_timestamp < {treatment_date}) max_fr_ts 
            from (
                  select rev_id, rev_page, rev_timestamp, page_namespace from revision_userindex
                    join page on page_id = rev_page where rev_user={user_id} 
                    and rev_timestamp < {treatment_date}
                    order by rev_timestamp desc limit 500) auser
            left join flaggedrevs on
                fr_page_id = rev_page and
                fr_rev_id = rev_id;
                    """.format(user_id=user_id, treatment_date=treatment_date.strftime('%Y%m%d%H%M%S'))
    con.execute('use dewiki_p;')
    rev_flag = pd.read_sql(rev_flag_sql, con)
    rev_flag['fr_timestamp'] = rev_flag['fr_timestamp'].apply(wmftimestamp)
    rev_flag['max_fr_ts'] = rev_flag['max_fr_ts'].apply(wmftimestamp)
    rev_flag['rev_timestamp'] = rev_flag['rev_timestamp'].apply(wmftimestamp)
    return rev_flag

def was_reverted(rev_id):
    try:
        _, reverted, reverted_to = mwreverts.api.check(
            de_session, rev_id, radius=3,  # most reverts within 5 edits
            window=48*60*60,  # 2 days
            rvprop={'user', 'ids'})  # Some properties we'll make use of
        return True if reverted else False
    except (KeyError, mwapi.session.APIError) as err:
        print('Error getting revert status for {rev_id}'.format(rev_id=rev_id))
        return True #because even if it was deleted from the DB for our purposes its still a bad edit 

def decide_flagged(row):
    """Was this revision flagged (or generally high quality)?"""
    namespace = row['page_namespace']
    rev_time = row['rev_timestamp']
    flagged_time = row['fr_timestamp']
    last_flagged_time = row['max_fr_ts']
    was_reverted = row['was_reverted']
    
    #namespace check
    if namespace != 0:
        return True # because often user-page edits are never approved   
    # check if explictly flagged
    elif pd.notnull(flagged_time):
        return True
    #check if reverted
    elif was_reverted:
        return False
    #check if the last flagged time is after the edit
    elif last_flagged_time:
        if rev_time < last_flagged_time:
            return True
        #the revision exists but it hasn't been flagged yet.
        else:
            return False
    else:
        #not sure what else would get to this stage , but...
        return False
        
def get_flagged_decision_df(user_id, treatment_date):
    rev_df = get_revisions_and_flagged_data(user_id, treatment_date)
    rev_df['was_reverted'] = rev_df.apply(lambda row: was_reverted(row['rev_id']) if pd.isnull(row['fr_timestamp']) else 'no_check', axis=1)
    rev_df['flagged'] = rev_df.apply(decide_flagged, axis=1)
    return rev_df


def get_flagged_revs(user_id, treatment_date, flagged_only_17=True):
    needed_columns = ['user_id', 'rev_id', 'rev_timestamp','was_flagged', 'was_reverted']
    rev_df = get_flagged_decision_df(user_id, treatment_date)
    rev_df['user_id'] = user_id
    rev_df['was_flagged'] = rev_df.apply(decide_flagged, axis=1)
    if flagged_only_17 == False:
        return rev_df
    #limit to only flagged
    rev_df = rev_df[rev_df['was_flagged']==True]
    # limit ot non-talk pages
    rev_df = rev_df[rev_df['page_namespace'].apply(lambda ns: ns%2==0)]
    # Limit to 17
    if rev_df.empty:
        return pd.DataFrame(columns=needed_columns)
    rev_df = rev_df.sort_values('rev_timestamp', ascending=False)
    rev_df = rev_df.iloc[:17]

    return rev_df[needed_columns]

In [2]:
langs = ['de']

langcode = langs[0]

datadir = os.path.join('data', langcode)


site = mwclient.Site(('https', f'{langcode}.wikipedia.org'), path = '/w/')

os.makedirs(datadir, exist_ok=True)

constr = 'mysql+pymysql://{user}:{pwd}@{host}/DB?charset=utf8'.format(user=os.environ['MYSQL_USERNAME'],
                                                      pwd=os.environ['MYSQL_PASSWORD'],
                                                      host=os.environ['MYSQL_HOST'])

con = create_engine(constr, encoding='utf-8')


con.execute(f'use dewiki_p;')

## What I understand about flagged revs
There are types (automatic, if the users is already an exeperience member, and non-automatic).

The table `flaggedrevs` has both of these types. However if the column "fr_flags" doesn't contain automatic, then it displays the time at which it was flagged, not when the underlying edit was made. 


Question. If an edit is not explicitly flagged, but a later one is flagged, does the intermediate one count as flagged? Yes, it seems. So how do we know if a non-flagged edit was in between? My working assumption is that to see this case, the edit would be  revereted.

So to use a table:

flagged edit (gesichtet): a) automatically or manually flagged b) a non-reverted edit, and the latest flagged edit is later than edit in question
unflagged edit: a) reverted b) not in flagged table and the latest flagged edit is before edit in question

and only namespace = 0

In [43]:
robrii

,rev_id,rev_timestamp,was_flagged,was_reverted
0,174729358,2018-03-05 20:58:32,True,no_check
1,174727397,2018-03-05 19:58:26,True,no_check
2,174727341,2018-03-05 19:56:16,True,no_check
3,174727130,2018-03-05 19:48:11,True,no_check
4,174726986,2018-03-05 19:42:23,True,no_check
5,174718915,2018-03-05 15:10:53,True,no_check
6,174718138,2018-03-05 14:45:03,True,no_check
7,174718133,2018-03-05 14:44:56,True,no_check
8,174718132,2018-03-05 14:44:55,True,no_check
9,174718127,2018-03-05 14:44:52,True,no_check


In [ ]:
robri[robri['was_reverted']==True]

JosefMüller_f = get_flagged_revs(2869700, datetime.datetime(2019,3,6))

In [7]:
JosefMüller_f

,rev_id,rev_timestamp,was_flagged
0,174736262,2018-03-06 06:19:22,True
1,174736268,2018-03-06 06:20:06,True
2,174736278,2018-03-06 06:20:47,True
3,174736301,2018-03-06 06:23:18,True
4,174736318,2018-03-06 06:24:34,True
5,174736586,2018-03-06 06:43:07,True
6,174736875,2018-03-06 07:03:16,True
7,174737096,2018-03-06 07:13:37,True
8,174737141,2018-03-06 07:15:42,True
9,174737343,2018-03-06 07:24:53,True
